### Initialize

In [535]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [52]:
tutorial ="https://realpython.com/beautiful-soup-web-scraper-python/"

In [538]:
df=pd.read_html("dd.html",encoding="utf-8")[0]
file=open("dd.html","r")


### BS4 Parser to DataFrame

In [539]:
soup=BeautifulSoup(file,"html.parser")
res=soup.find_all("i",class_="fa fa-info-circle fa-lg")
k={}
po=[]
pil=0
fil=0
for i in res:
    k={}
    test=i
    test=test.attrs["onclick"]
    rest=re.search("\((.+)\)",test).groups()[0][1:-1]
    rest=str(rest).replace("<br>","")
    soop=BeautifulSoup(rest,"html.parser")
    for i in range(0,len(soop.contents),2):

        lanza=str(soop.contents[i]).strip()
        lanza=lanza.replace("<b>","")
        lanza=lanza.replace("</b>","")
        lanza=lanza.replace(":","")
        lanza=lanza.replace("-","")
        lanza=lanza.strip()
        lanza=str(lanza)
        fanza=str(soop.contents[i+1]).strip()
        k.update({lanza:fanza})
    if(len(k)>pil):
        pil=len(k)
        fil=k
    po.append(k)
    # fef=pd.Series(k.values())
    # ref.loc[len(ref)]=fef
ref=pd.DataFrame(columns=list(fil.keys()))
for i in po:
    fef=pd.Series(i.values(),index=list(i.keys()))
    ref.loc[len(ref)]=fef

### Merging & **DataScience**

In [540]:
df=pd.concat([df,ref],axis=1).head(170)
df.drop(["Unnamed: 11"],axis=1,inplace=True)
pef=df.copy()

In [541]:
def corrector(name:str):
    res=re.findall(r"\d+",name)[0]
    res=str(res).strip()
    res=int(res)
    return res
def exam_ex(name:str):
    name=str(name).strip()
    # print(name)
    res=re.search(r"(\w+) ساعت (\d+)",name).groups()
    return (res[0],int(res[-1]))
def faculty_ex(name):
    try:
        res=re.search(r"کلاس در دانشکده (.+) برگزار میشود",name).groups()[0]
        return res
    except:
        return np.NaN
def fully(name:str):
    d=name.split("/")
    b=int(d[0])
    c=int(d[-1])
    if(b==c):
        return True
    else:
        return False

In [542]:
pef["Male"]=pef["قابل انتخاب برای دانشجویان"].transform(lambda x: True if "مرد" in str(x) else False)
pef["Valid_Year"]=pef["قابل انتخاب برای دانشجویان"].transform(lambda x: corrector(x))
pef["Exam_day"]=pef["امتحان روز"].transform(lambda x: str(exam_ex(x)[0]).split())
pef["Exam_time"]=pef["امتحان روز"].transform(lambda x: exam_ex(x)[-1])
pef.drop(["امتحان روز","زبان ارائه","دانشکده","مقطع","نحوه ارائه","قابل انتخاب برای دانشجویان"],inplace=True,axis=1)
pef.drop(pef[~pef["جلسه دوم روز"].isna()].index,axis=0,inplace=True)
pef=pef[pef["واحد"]==2]
pef.rename(columns={"جلسه اول روز":"Class Time"},inplace=True)
pef["Class Time"]=pef["Class Time"].apply(lambda x: str(exam_ex(x)[0])+"_"+str(exam_ex(x)[-1])  )
pef.drop(["جلسه دوم روز"],inplace=True,axis=1)
pef=pef[(pef["واحد"]==2) & (pef["Male"]==True)]
pef.index=pef["ردیف"]
pef.drop(["Male","واحد","ردیف"],inplace=True,axis=1)
pef.rename(columns={"شرح درس":"Faculty"},inplace=True)
pef["Faculty"]=pef["Faculty"].apply(lambda x:faculty_ex(x))
pef.drop(pef[pef["Faculty"].isna()].index,axis=0,inplace=True)
pef.rename(columns={"ثبت نام شده":"Register_Status"},inplace=True)
pef["Register_Status"]=pef["Register_Status"].astype("str")
pef["ظرفیت"]=pef["ظرفیت"].astype("str")
pef["Register_Status"]=pef["Register_Status"]+"/"+pef["ظرفیت"]
pef.drop(["ظرفیت"],inplace=True,axis=1)
Shomareh=pef.drop(["شماره درس"],axis=1)
pef.drop(["شماره درس"],inplace=True,axis=1)
pef["Full"]=pef["Register_Status"].transform(lambda x:fully(x))
pef.shape

(61, 12)

In [545]:
df=pef.copy()

### Finally

In [554]:
f=df.sort_values(by=["رزرو"],ascending=False)[["نام استاد","رزرو"]]

In [556]:
f

,نام استاد,رزرو
ردیف,,
46,سید علی دریاباری,21
157,یاسر دلشاد,17
161,رضا مرشدلو,15
45,سید علی دریاباری,11
56,یاسر دلشاد,11
...,...,...
128,NaN,0
123,NaN,0
122,عباسعلی نصر آبادی,0
